# Abstract for the Project

## Task

## Model

## Experiment

## Datasets

# Code

## Import statements

In [1]:
!pip install -U tensorflow-text==2.8.1

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import neural_structured_learning as nsl
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

# Resets notebook state
tf.keras.backend.clear_session()

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print(
    "GPU is",
    "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.8.1
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


In [ ]:
from dataLoader import *
from bertEmbeddings import *
from tfRecordTools import *
from train import *

## Dataset Loading

In [3]:
# Load data from IMDB
imdb = tf.keras.datasets.imdb
(pp_train_data, pp_train_labels), (pp_test_data, pp_test_labels) = (
    imdb.load_data(num_words=10000))

In [4]:
# This block limits how much is loaded to keep debugging short
# don't run on the final experiments

limit = 1000

pp_train_data = pp_train_data[0:limit]
pp_train_labels = pp_train_labels[0:limit]
pp_test_data = pp_test_data[0:limit]
pp_test_labels = pp_test_labels[0:limit]

In [5]:
# Print data entries
print('Training entries: {}, labels: {}'.format(
    len(pp_train_data),
    len(pp_train_labels)
))
training_samples_count = len(pp_train_data)
print(pp_train_data[0])
len(pp_train_data[0]), len(pp_train_data[1])

Training entries: 1000, labels: 1000
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


(218, 189)

In [7]:
decode_review(pp_train_data[0], imdb)

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

## Pretrained Embeddings

In [8]:
# Retrieve small bert embeddings
pretrained_embedding = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2'

In [9]:
# Preprocess and encode input
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
encoder_inputs = preprocessor(text_input)

encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2',trainable=True)

outputs = encoder(encoder_inputs)

pooled_output = outputs['pooled_output'] # [batch_size, 128].
# [batch_size, seq_length, 128].

sequence_output = outputs['sequence_output']

In [ ]:
createBertEmbeddingExample(pp_train_data[0], 0, imdb, encoder, preprocessor)

In [ ]:
# Generate full BERT embeddings

#bertEmbeddingsPath = '/tmp/imdb/bertEmeddings.tfr'
#createBertEmbedding(pp_train_data, bertEmbeddingsPath, 0, imdb, encoder, preprocessor)

## Training and Validation

### Training Hyperparameters

In [26]:
# Set parameters
NBR_FEATURE_PREFIX = 'NL_nbr_'
NBR_WEIGHT_SUFFIX = '_weight'
HPARAMS = HParams()

### Sample Features

In [ ]:
# Persist TF.Example features (word vectors and labels) for training and test
# data in TFRecord format.
next_record_id = create_records(pp_train_data,
                                pp_train_labels,
                                '/tmp/imdb/train_data.tfr',
                                0)
create_records(pp_test_data,
               pp_test_labels,
               '/tmp/imdb/test_data.tfr',
               next_record_id)

### Build Traininig and Testing Datasets

In [ ]:
# Create training and testing datasets
train_dataset = make_dataset('/tmp/imdb/nsl_train_data.tfr', HPARAMS, NBR_FEATURE_PREFIX, NBR_WEIGHT_SUFFIX, True)
test_dataset = make_dataset('/tmp/imdb/test_data.tfr', HPARAMS, NBR_FEATURE_PREFIX, NBR_WEIGHT_SUFFIX, False)

### Build Validation Set

In [ ]:
validation_fraction = 0.9
validation_size = int(validation_fraction *
                      int(training_samples_count / HPARAMS.batch_size))
print(validation_size)
validation_dataset = train_dataset.take(validation_size)
train_dataset = train_dataset.skip(validation_size)

## Setting up Base Models (Bi-LSTM)

### Create Models

In [31]:
# build Bi-LSTM model
model = make_bilstm_model(HPARAMS)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 words (InputLayer)          [(None, 256)]             0         
                                                                 
 embedding (Embedding)       (None, 256, 16)           160000    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              41472     
 l)                                                              
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 209,793
Trainable params: 209,793
Non-trainable params: 0
_______________________________________________________

In [32]:
# set loss function and optimizer
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy'])

### Fit Model

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=HPARAMS.train_epochs,
    verbose=1)

### Evaluate Model

In [ ]:
# evaluate model performance on test data
results = model.evaluate(test_dataset, steps=HPARAMS.eval_steps)
print(results)

### Plot Performance

In [ ]:
# plot training and validaiton loss over epochs
history_dict = history.history
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "-r^" is for solid red line with triangle markers.
plt.plot(epochs, loss, '-r^', label='Training loss')
# "-b0" is for solid blue line with circle markers.
plt.plot(epochs, val_loss, '-bo', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='best')

plt.show()

In [ ]:
# plot training and validation accuracy over epochs
plt.clf()   # clear figure

plt.plot(epochs, acc, '-r^', label='Training acc')
plt.plot(epochs, val_acc, '-bo', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='best')

plt.show()

## CNN Models

### Same categories as before

## Graph Models

### Same categories as before